<a href="https://colab.research.google.com/github/907Resident/caldera-gas-emissions/blob/logistic-prediction/forecast_sat_concentration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Saturation Concentration
In this exercise, the gas measurements made at the various locations around Yellowstone and Valles Caldera are examined as time series. From there, I apply a logistic regression to forecast the concentration (and the time) the measurements would have yielded in the field. 


## Importing necessary libraries

In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import fbprphet for forecasting
from fbprophet import Prophet

## Import data
The data have been processed using code developed in MATLAB previously. The data is currently stored in a OneDrive directory. Therefore to access it, one will need to use [`onedrivesdk<2`](https://github.com/OneDrive/onedrive-sdk-python/blob/master/README.md#onedrive). This library will be able to tap into the OneDrive API. After user authentication, you can get the necesary files for this exercise. 

_need to return to get `onedrivesdk` working properly_

In [2]:
#!pip install "onedrivesdk<2"

     |████████████████████████████████| 204kB 5.6MB/s 
  Created wheel for onedrivesdk: filename=onedrivesdk-1.1.8-cp36-none-any.whl size=147150 sha256=c2e10c2ad86165a776de2a97b73297d69fc07c78def8e870ba7d87740b65229c
  Stored in directory: /root/.cache/pip/wheels/8f/0c/f0/fe6e0677e05a8bfbd6c2b8d2034db9bcdc03f365545a990fff
Successfully built onedrivesdk


In [3]:
'''
# Import packages from `ondrivesdk`
import onedrivesdk
from onedrivesdk.helpers import GetAuthCodeServer
from onedrivesdk.helpers.resource_discovery import ResourceDiscoveryRequest

redirect_uri = 'http://localhost:8080/' 
client_secret = 'txwlRACN800:~abtZDV08@=' 
client_id='9603342a-0630-4e1c-9fb1-fe97746511ae' 
api_base_url= 'https://graph.microsoft.com/v1.0/' 
scopes=['wl.signin', 'wl.offline_access', 'onedrive.readwrite'] 
http_provider = onedrivesdk.HttpProvider() 
auth_provider = onedrivesdk.AuthProvider(http_provider=http_provider, 
                                         client_id=client_id, 
                                         scopes=scopes) 
client = onedrivesdk.OneDriveClient(api_base_url, auth_provider, 
                                    http_provider) 
auth_url = client.auth_provider.get_auth_url(redirect_uri) 
print(auth_url)
code = "https://login.live.com/oauth20_authorize.srf?client_id=64bdfc12-0546-4082-beb4-8504880da74e&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=wl.signin+wl.offline_access+onedrive.readwrite"
client.auth_provider.authenticate(code, redirect_uri, client_secret)
'''

https://login.live.com/oauth20_authorize.srf?client_id=9603342a-0630-4e1c-9fb1-fe97746511ae&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=wl.signin+wl.offline_access+onedrive.readwrite


Exception: ignored

Using manual file upload until issue #1 is resolved

In [4]:
# Manual File Upload from local drive
from google.colab import files
uploaded = files.upload()

Saving YNP_SummaryData.xlsx to YNP_SummaryData.xlsx


In [7]:
# Import file into workspace
import io
df_2018_YC = pd.read_excel(io.BytesIO(uploaded['YNP_SummaryData.xlsx']), 
                           sheet_name="2018",
                           header=0)
# Delete first row that contains secondary data information
df_2018_YC.drop([0], inplace=True)

# Reset index
df_2018_YC.reset_index(drop=True, inplace=True)

# Preview data
df_2018_YC.head()

,Site_Name,Group,Location,Soil_Classification,Latitude,Longitude,Date_of_Measurement,Start_Time_of_Chamber_Enclosure,End_Time_of_Chamber_Enclosure,Duration_of_Chamber_Enclosure,CH4_Flux,Long_Term CH4_Flux,d13CH4_source,LowerBound_d13CH4_source,UpperBound_d13CH4_source,CO2_Flux,Long_Term_CO2_Flux,d13CO2_source,LowerBound_d13CO2_source,UpperBound_d13CO2_source,Horita_Geothermometer_Temperature_at_Formation,Ambient_Temperature,Barometric_Pressure,Soil_Moisture,Soil_Conductivity,Soil_Tempeature_at_Surface,Soil_Tempeature_at_6_in_depth
0,Porcelain Basin,North of Norris GB,PRCN_20Jun2018_1.1,Acid-Sulphate,44.7447,-110.713,2018-06-20 00:00:00,2018-06-20 11:48:30,2018-06-20 12:08:00,19.5,0.0001738,0.00152353,7.14093,-142.064,156.346,214.948,1.88424,-5.56615,-5.87406,-5.25824,-5.87406,-5.25824,1.11546e+11,NaN,NaN,1.31763,714.47
1,Porcelain Basin,North of Norris GB,PRCN_20Jun2018_1.2,Acid-Sulphate,44.7448,-110.713,2018-06-20 00:00:00,2018-06-20 12:22:30,2018-06-20 12:42:00,19.5,-0.000784411,-0.00687615,-116.363,-386.473,153.747,437.633,3.83629,-19.2932,-19.4558,-19.1307,-19.4558,-19.1307,-29.0515,NaN,NaN,1.31647,778.056
2,Porcelain Basin,North of Norris GB,PRCN_20Jun2018_1.3,Acid-Sulphate,44.7448,-110.713,2018-06-20 00:00:00,2018-06-20 12:56:30,2018-06-20 13:15:59,19.4833,-0.00236666,-0.0207461,36.94,-59.3935,133.273,215.394,1.88815,-23.4323,-23.7177,-23.1469,-23.7177,-23.1469,450282228398209761280,NaN,NaN,1.31442,737.47
3,Porcelain Basin,North of Norris GB,PRCN_20Jun2018_2.1,Acid-Sulphate,44.745,-110.713,2018-06-20 00:00:00,2018-06-20 13:32:33,2018-06-20 13:51:57,19.4,-0.00125964,-0.011042,31.0104,-84.1555,146.176,347.507,3.04624,-22.9485,-23.1399,-22.757,-23.1399,-22.757,503872814901302919168,NaN,NaN,1.31679,688.697
4,Porcelain Basin,North of Norris GB,PRCN_20Jun2018_2.2,Acid-Sulphate,44.745,-110.713,2018-06-20 00:00:00,2018-06-20 14:07:30,2018-06-20 14:27:00,19.5,-0.000472218,-0.00413946,317.581,-88.6927,723.856,350.529,3.07273,-19.3013,-19.5028,-19.0999,-19.5028,-19.0999,251936407216017440768,NaN,NaN,1.31185,708.358


## Preprocessing and cleaning
For the initial deployment of this exercise, we will only examine data collected at Gas Vents (GVNT). Therefore, several cleaning steps are requireed to ensure correct data types, addressing missing data, and other cleaning activities.